In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.compat.v2 as tf2
import tensorflow_datasets as tfds
import datetime

In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

In [32]:
encoder = info.features['text'].encoder
buffer_size = 1000
batch_size = 10
train_batches = train_data.shuffle(buffer_size).padded_batch(batch_size,drop_remainder=True)
test_batches = test_data.shuffle(buffer_size).padded_batch(batch_size,drop_remainder=True)
print(encoder.subwords[-20:])
(list(train_batches.as_numpy_iterator()))

['algi', 'adventures', 'accurate_', 'accompan', 'Thom', 'Still_', 'Someone', 'Serious', 'SU', 'Phill', 'Perso', 'Patrick_', 'Lei', 'Jus', 'Gho', 'Get_', 'Freeman', 'Especially_', '?).', '..."']


[(array([[ 768,  284,    1, ..., 6873, 1017, 7975],
         [  19,  118, 1327, ...,    0,    0,    0],
         [  12,  674,   55, ...,    0,    0,    0],
         ...,
         [1375,    6,    1, ...,    0,    0,    0],
         [ 133,   67, 3698, ...,    0,    0,    0],
         [ 249,    4,  301, ...,    0,    0,    0]]),
  array([1, 0, 1, 1, 1, 1, 1, 0, 0, 0])),
 (array([[ 147,    1, 5302, ...,    0,    0,    0],
         [5265, 2733,  871, ...,    0,    0,    0],
         [1284,  405, 1055, ...,    0,    0,    0],
         ...,
         [ 373,    6,    1, ...,    0,    0,    0],
         [6553, 1447,  363, ..., 3896, 5444, 7975],
         [  19,   32, 1422, ...,    0,    0,    0]]),
  array([1, 0, 1, 0, 1, 0, 0, 1, 1, 1])),
 (array([[  12,  284,    1, ...,    0,    0,    0],
         [3186, 2528, 1113, ...,    0,    0,    0],
         [ 387,  769,    9, ..., 1870,  824, 7975],
         ...,
         [  12,  641,    7, ...,    0,    0,    0],
         [5646,    2,   14, ...,    0,

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 16),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
print(model.summary())
log_dir = "/home/manju/code/ML/src/manju_tensorflow_examples/logs/fit/" \
 + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + 'word_embedding_rnn_emb_16_lstm_16_lstm_16_d_16_d_1_adam_lr_0001'
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=10,
                    validation_data=test_batches, 
                    validation_steps=30,
                    callbacks=[tb_callback])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 16)          130960    
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 16)          2112      
_________________________________________________________________
dense_20 (Dense)             (None, None, 16)          272       
_________________________________________________________________
dense_21 (Dense)             (None, None, 1)           17        
Total params: 133,361
Trainable params: 133,361
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
390/390 [==============================] - 165s 418ms/step - loss: 0.6931 - accuracy: 0.4986 - val_loss: 0.6927 - val_accuracy: 0.5250
Epoch 2/10
390/390 [==============================] - 167s 427ms/step - loss: 0.6926 - accurac

In [23]:
test_loss, test_acc = model.evaluate(test_batches)

390/390 [==============================] - 29s 75ms/step - loss: 0.5233 - accuracy: 0.7847
